• The pitfalls of subclassing from built-in types.  
• Multiple inheritance and the method resolution order.

# 1. Subclassing built-in types in tricky

In [1]:
# Ex 12-1. Our __setitem__ override is ignored by 
# the __init__ and __update__ methods of the built-in dict.
class DoppleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [3]:
dd = DoppleDict(one=1)
dd

{'one': 1}

In [4]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [6]:
dd.update(three=3)
dd

{'one': 1, 'three': 3, 'two': [2, 2]}

In [7]:
# Ex 12-2. The __getitem__ of AnswerDict is byepaseed by dict.update
class AnswerDict(dict):
    def __getitem__(self, key):
        return 32

In [9]:
ad = AnswerDict(a='foo')
ad['a']

32

In [10]:
d = {}
d.update(ad)
d['a']

'foo'

In [11]:
d

{'a': 'foo'}

In [20]:
# Ex 12-3. DoppleDict2 and AnswerDict2 work as expected
# because they extend UserDict and not dict
import collections
class DoppleDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [21]:
dd = DoppleDict2(one=1)
dd

{'one': [1, 1]}

In [22]:
dd['two'] = 2
dd

{'two': [2, 2], 'one': [1, 1]}

In [23]:
dd.update(three=3)
dd

{'two': [2, 2], 'three': [3, 3], 'one': [1, 1]}

In [24]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [25]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [26]:
d = {}
d.update(ad)
d['a'], d

(42, {'a': 42})

If you subclass from a class coded in
Python, such as UserDict or MutableMapping, you will not be troubled by this

# 2. Multiple inheritance and method resolution ordera

In [27]:
# Ex 12-4 diamond.py: class A, B, C and D form the graph
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [28]:
d = D()
d.pong()

pong: <__main__.D object at 0x1065ac1d0>


In [29]:
C.pong(d)

PONG: <__main__.D object at 0x1065ac1d0>


In [30]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [31]:
d.pingpong()

ping: <__main__.D object at 0x1065ac1d0>
post-ping: <__main__.D object at 0x1065ac1d0>
ping: <__main__.D object at 0x1065ac1d0>
pong: <__main__.D object at 0x1065ac1d0>
pong: <__main__.D object at 0x1065ac1d0>
PONG: <__main__.D object at 0x1065ac1d0>


In [34]:
# Ex 12-8 Inpsecting the __mro__ atribute in several classes

In [35]:
bool.__mro__

(bool, int, object)

In [36]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [37]:
print_mro(bool)

bool, int, object


In [38]:
import io

In [39]:
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [40]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


# 3. Multiple inheritance in the real word

In [32]:
import tkinter

In [42]:
print_mro(tkinter.Toplevel)
print_mro(tkinter.Widget)
print_mro(tkinter.Button)
print_mro(tkinter.Entry)
print_mro(tkinter.Text)

Toplevel, BaseWidget, Misc, Wm, object
Widget, BaseWidget, Misc, Pack, Place, Grid, object
Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object
Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object
Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


# 4. Coping with multiple inheritance

1. Distinguish interface inheritance from implementation inheritance  
2. Make interfaces explicit with ABCs
3. Use mixins for code reuse
4. Make mixins explicit by naming
5. An ABC may also be a mixin; the reverse is not true
6. Don's subclass from more than one concrete class
7. Provide aggregate classes to users
8. Favor object composition over class inheritance
9. Tkinter: the good, the bad and the ugly

# 5. A modern example: mixins in Django generic views